In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os

import numpy as np
import pandas as pd
import atecml.data
from tqdm import tqdm

In [2]:
#训练集为第一步build的纬度提升矩阵，并过滤掉unknown标签
train_df = pd.read_pickle('./01_train.dat')
val_df =  pd.read_pickle('./01_test.dat')

#train_df = atecml.data.load_train()
#val_df = atecml.data.load_test()
train_df.loc[train_df.label == 0, 'Fraud'] = 0
train_df.loc[train_df.label != 0, 'Fraud'] = 1


In [3]:
import joblib
#predictors =joblib.load('./woe_feature.dat')
predictors = [x for x in train_df.columns if x not in atecml.data.NOT_FEATURE_COLUMNS]


target = 'Fraud'
DateFold={}

DateFold[0] = set(atecml.data.filter_date(train_df,start_date='2017-09-05',end_date='2017-09-12').index)
DateFold[1] = set(atecml.data.filter_date(train_df,start_date='2017-09-13',end_date='2017-09-20').index)
DateFold[2] = set(atecml.data.filter_date(train_df,start_date='2017-09-21',end_date='2017-09-28').index)
DateFold[3] = set(atecml.data.filter_date(train_df,start_date='2017-09-29',end_date='2017-10-06').index)
DateFold[4] = set(atecml.data.filter_date(train_df,start_date='2017-10-07',end_date='2017-10-14').index)
DateFold[5] = list(atecml.data.filter_date(train_df,start_date='2017-10-15',end_date='2017-11-24').index)

all_list = set(train_df.index) - set(DateFold[5])
len(all_list),len(DateFold[5])

(634284, 360447)

In [10]:
X = np.array(train_df[predictors])
Y = np.array(train_df[target])

val_index = DateFold[5] 
train_index = list(DateFold[3]) + list(DateFold[4])

X_train = X[train_index]
y_train = Y[train_index]

X_val = X[val_index]
y_val = Y[val_index]
    
X_test = np.array(val_df[predictors])


In [11]:
print(X_train.shape, y_train.shape)

(250554, 466) (250554,)


In [12]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,free_raw_data=False)

print('设置参数')
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'binary_logloss',
          }

### 交叉验证(调参)
print('交叉验证')
min_merror = float('Inf')
best_params = {}

# 准确率
print("调参1：提高准确率")
for num_leaves in range(20,200,10):
    for max_depth in range(3,10,3):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth

        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            num_boost_round=500,
                            seed=2018,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
            
        mean_merror = pd.Series(cv_results['auc-mean']).min()
        boost_rounds = pd.Series(cv_results['auc-mean']).argmin()
            
        if mean_merror < min_merror:
            min_merror = mean_merror
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth
            
params['num_leaves'] = best_params['num_leaves']
params['max_depth'] = best_params['max_depth']

设置参数
交叉验证
调参1：提高准确率
[1]	cv_agg's auc: 0.750856 + 0.06993
[2]	cv_agg's auc: 0.933385 + 0.0037631
[3]	cv_agg's auc: 0.950871 + 0.00274064
[4]	cv_agg's auc: 0.956118 + 0.0019865
[5]	cv_agg's auc: 0.961227 + 0.00267791
[6]	cv_agg's auc: 0.96355 + 0.00279506
[7]	cv_agg's auc: 0.964781 + 0.00311122
[8]	cv_agg's auc: 0.965972 + 0.0030792
[9]	cv_agg's auc: 0.966619 + 0.00267428
[10]	cv_agg's auc: 0.967899 + 0.00274367
[11]	cv_agg's auc: 0.968664 + 0.00251366
[12]	cv_agg's auc: 0.969584 + 0.00273821
[13]	cv_agg's auc: 0.970182 + 0.00222686
[14]	cv_agg's auc: 0.970817 + 0.00203878
[15]	cv_agg's auc: 0.971227 + 0.00189825
[16]	cv_agg's auc: 0.971499 + 0.00190475
[17]	cv_agg's auc: 0.971976 + 0.00174649
[18]	cv_agg's auc: 0.972206 + 0.00190165
[19]	cv_agg's auc: 0.972618 + 0.00188956
[20]	cv_agg's auc: 0.972925 + 0.00188168
[21]	cv_agg's auc: 0.973321 + 0.00181199
[22]	cv_agg's auc: 0.973652 + 0.00168735
[23]	cv_agg's auc: 0.973848 + 0.00170901
[24]	cv_agg's auc: 0.973931 + 0.00168504
[25]	cv_agg'

KeyboardInterrupt: 

In [5]:
num_pos = np.sum(train_df[target])  
num_neg = x_train.shape[0]- num_pos
scale_pos_weight =  num_neg/num_pos
print(num_pos,num_neg,scale_pos_weight)

16847.0 977884.0 58.04499317385885


In [ ]:
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,free_raw_data=False)

In [ ]:
num_boost_round = 2000
num_early_stop = 100

In [ ]:

dart_classifier = LGBClassifier(boost_type='dart',boost_round=num_boost_round,early_stop=num_early_stop,pos_weight= scale_pos_weight)
dart_oof_train, dart_oof_test,dart_model_list = dart_classifier.get_oof(x_train, y_train, x_test)
print(dart_oof_train.shape, dart_oof_test.shape)   


gbdt_classifier = LGBClassifier(boost_type='gbdt',boost_round=num_boost_round,early_stop=num_early_stop,pos_weight= scale_pos_weight)
gbdt_oof_train, gbdt_oof_test,gbdt_model_list = gbdt_classifier.get_oof(x_train, y_train, x_test)
print(gbdt_oof_train.shape, gbdt_oof_test.shape)  
 


{'min_child_samples': 20, 'metric': {'auc'}, 'num_leaves': 80, 'subsample': 0.85, 'task': 'train', 'nthread': 40, 'is_unbalance': True, 'learning_rate': 0.05, 'objective': 'binary', 'subsample_freq': 1, 'subsample_for_bin': 2000, 'boosting_type': 'dart', 'verbose': -1, 'use_missing': 'true', 'min_split_gain': 0, 'max_bin': 511, 'colsample_bytree': 0.7, 'max_depth': -1}
0 fold, train 500685, val 360447
train with lgb model
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.967054
[100]	valid_0's auc: 0.97075
[150]	valid_0's auc: 0.973024
[200]	valid_0's auc: 0.974198
[250]	valid_0's auc: 0.975055
[300]	valid_0's auc: 0.975691
[350]	valid_0's auc: 0.976046
[400]	valid_0's auc: 0.976443
[450]	valid_0's auc: 0.97675
[500]	valid_0's auc: 0.976812
[550]	valid_0's auc: 0.976832
[600]	valid_0's auc: 0.977054
[650]	valid_0's auc: 0.977077
[700]	valid_0's auc: 0.977027
[750]	valid_0's auc: 0.976731
Early stopping, best iteration is:
[690]	valid_0's auc: 0.977111

In [ ]:
train_dart_r = gbdt_oof_train
n = int(gbdt_oof_train.shape[0] * 0.8)
y_pred, y_val = train_dart_r[n:], y_train[n:]
_f1,_f2,_f3 = atecml.data.accuracy_validation(y_val,y_pred)


In [ ]:
train_dart_r = dart_oof_train
n = int(dart_oof_train.shape[0] * 0.8)
y_pred, y_val = train_dart_r[n:], y_train[n:]
_f1,_f2,_f3 = atecml.data.accuracy_validation(y_val,y_pred)